## Multi-Accent and Multi-Lingual Voice Clone Demo with MeloTTS

In [1]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

/opt/homebrew/anaconda3/envs/openvoicev2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



### Initialization

In this example, we will use the checkpoints from OpenVoiceV2. OpenVoiceV2 is trained with more aggressive augmentations and thus demonstrate better robustness in some cases.

In [3]:
ckpt_converter = 'checkpoints/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs_v2'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

Loaded checkpoint 'checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []


### Obtain Tone Color Embedding
We only extract the tone color embedding for the target speaker. The source tone color embeddings can be directly loaded from `checkpoints_v2/ses` folder.

In [4]:

reference_speaker = 'resources/demo_speaker2.mp3' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=False)

OpenVoice version: v1


Estimating duration from bitrate, this may be inaccurate


#### Use MeloTTS as Base Speakers

MeloTTS is a high-quality multi-lingual text-to-speech library by @MyShell.ai, supporting languages including English (American, British, Indian, Australian, Default), Spanish, French, Chinese, Japanese, Korean. In the following example, we will use the models in MeloTTS as the base speakers. 

In [5]:
from melo.api import TTS
import torch

output_dir = "outputs_v2"  # Replace with your actual output directory
device = "cuda" if torch.cuda.is_available() else "cpu"

text = "The integration of AI into various industries has driven significant advancements in hardware technology. PC manufacturers are now offering AI-specific hardware to enhance end-user devices. Field-Programmable Gate Arrays (FPGAs) and Application-Specific Integrated Circuits (ASICs), such as Google's Tensor Processing Units (TPUs), are being utilized to accelerate machine learning workloads. These hardware solutions are particularly useful in edge computing scenarios, where AI capabilities need to be deployed close to the data source to reduce latency and improve decision-making. Microsoft has introduced Copilot+ PCs, which feature new silicon capable of performing over 40 trillion operations per second, enhancing AI capabilities such as real-time image generation and language translation. Apple has also integrated Apple silicon to handle advanced AI processing. These developments reflect the growing trend of AI processing moving closer to the user and the edge, rather than relying solely on cloud-based solutions."
src_path = f"{output_dir}/tmp.wav"
speed = 1.0

model = TTS(language="EN", device=device)
speaker_ids = model.hps.data.spk2id

# Print available speaker keys
print("Available speaker keys:", list(speaker_ids.keys()))

# Try to find an appropriate English speaker
english_speakers = [key for key in speaker_ids.keys() if key.startswith("EN")]

if not english_speakers:
    raise ValueError(
        "No English speaker found. Available speakers: " + ", ".join(speaker_ids.keys())
    )

# Use the first available English speaker
speaker_key = english_speakers[0]
speaker_id = speaker_ids[speaker_key]

print(f"Using speaker: {speaker_key}")

source_se = torch.load(
    f'checkpoints_v2/base_speakers/ses/{speaker_key.lower().replace("_", "-")}.pth',
    map_location=device,
)
model.tts_to_file(text, speaker_id, src_path, speed=speed)
save_path = f'{output_dir}/output_v2_{speaker_key.lower().replace("_", "-")}.wav'

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path,
    src_se=source_se,
    tgt_se=target_se,
    output_path=save_path,
    message=encode_message,
)

Available speaker keys: ['EN-US', 'EN-BR', 'EN_INDIA', 'EN-AU', 'EN-Default']
Using speaker: EN-US
 > Text split to sentences.
The integration of AI into various industries has driven significant advancements in hardware technology. PC manufacturers are now offering AI-specific hardware to enhance end-user devices. Field-Programmable Gate Arrays FPGAs and Application-Specific Integrated Circuits ASICs,
such as Google's Tensor Processing Units TPUs, are being utilized to accelerate machine learning workloads. These hardware solutions are particularly useful in edge computing scenarios, where AI capabilities need to be deployed close to the data source to reduce latency and improve decision-making.
Microsoft has introduced Copilot+ PCs, which feature new silicon capable of performing over 40 trillion operations per second, enhancing AI capabilities such as real-time image generation and language translation. Apple has also integrated Apple silicon to handle advanced AI processing.
These 

  0%|          | 0/4 [00:00<?, ?it/s]Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 4/4 [00:20<00:00,  5.06s/it]
